<a href="https://colab.research.google.com/github/jpedro-rod/ElementosFinitos/blob/main/Elementos_Finitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elementos Finitos 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd

Formulação forte de um problema de condução de calor estacionário em uma placa:

$\nabla\cdot\vec{q} + G = 0 \;,\;\; \vec{q} = -k\nabla T$

Formulação fraca:

$∫_{\Omega}[\nabla\cdot\vec{q} + G]\cdot \omega(x,y) \;\; d\Omega = 0, ∀\omega \in U^0$

$∫_{\Omega}[\nabla\cdot\vec{q}\omega + G\omega] \;\; d\Omega = 0, ∀\omega \in U^0$

Pelo Teorema de Green (funcionando como a regra do produto funcionava em 01 dimensão):

$\oint_{\Gamma}\omega\vec{q}\cdot\vec{n} \; d\Gamma - \int_{Ω} \nabla\omega\cdot\vec{q} \;\; dΩ + \int_{\Omega} G\omega \;\; dΩ= 0 \;\;\;\; (I)$

Reescrevendo a 1ª parcela em função das condições de contorno, ter-se-á:

$\oint_{\Gamma}\omega\vec{q}\cdot\vec{n} \; d\Gamma = \int_{\Gamma_q} \omega\vec{q}\cdot\vec{n} \; d\Gamma_q + \int_{\Gamma_T} \omega\vec{q}\cdot\vec{n} \; d\Gamma_T$

Sabe-se ainda que $\omega(x,y)=0$ em $\Gamma_T$, o que simplifica a equação oriunda do Teorema de Green:

$\oint_{\Gamma}\omega\vec{q}\cdot\vec{n} \; d\Gamma = \int_{\Gamma_q} \omega\vec{q}\cdot\vec{n} \; d\Gamma_q \;\;\;\; (II)$

Inserindo $II$ em $I$ e definindo $\overline{q}=\vec{q}\cdot\vec{n}$ finalmente:

$\int_{Ω} \nabla\omega\cdot\vec{q} \;\; dΩ = \int_{\Gamma_q} \omega\overline{q} \;\; dΓ_q + \int_{\Omega} \omega G \;\; dΩ$

$\int_{Ω} \nabla\omega\cdot(-k\nabla T) \;\; dΩ = \int_{\Gamma_q} \omega\overline{q} \;\; dΓ_q + \int_{\Omega} \omega G \;\; dΩ$

Discretizando a fórmula fraca:

$\int_{Ω} -B^{e^T}\omega^{e^T} \cdot k \cdot B^eT^e dΩ^e = \int_{\Gamma_q} N^{e^T}\omega^{e^T}\overline{q} \;\; dΓ_q^e + \int_{\Omega} N^{e^T}\omega^{e^T}G \;\; dΩ^e$

Onde $N^e$ é *matriz* de funções de interpolação (ou de forma) e $B^e$ é matriz das derivadas de $N^e$

$\int_{Ω} -B^{e^T} k B^e \cdot T^e dΩ^e = \int_{\Gamma_q} N^{e^T}\overline{q} \;\; dΓ_q^e + \int_{\Omega} N^{e^T}G \;\; dΩ^e$

## Caso em uma dimensão:

Condução de calor numa barra fina e homogênea com extremidades a temperaturas fixas e submetida a geração de calor de comportamento linear:

$L = 4\:m = 4 \;,\;\; A = 1\:m^2 \;,\;\; K = 5\:W/(°Cm)$

$G(x) = \frac{50}{3}(x+2) \;,\;\; T(x=1) = 100°C \;,\;\; T(x=4) = 0$

$\int_{Ω} B^{e^T} k B^e \; dx^e \cdot T^e + \int_{\Omega} N^{e^T}G \;\; dx^e = 0$

Funções de forma (ou de interpolação)

In [3]:
#1ª ordem:
#N1 = [N11, N12]

def N11(x):
    return 4-x

def N12(x):
    return x-3

def B11(x):
    return -1

def B22(x):
    return 1

#2ª ordem:
#N2 = [N21, N22, N23]

def N21(x):
    return (2/2**2)*(x-2)*(x-3)

def N22(x):
    return (-4/2**2)*(x-1)*(x-3)

def N23(x):
    return (2/2**2)*(x-1)*(x-2)

# B2 = [B21, B22, B23]

def B21(x):
    return (1/2)*(2*x-5)

def B22(x):
    return -2*x+4

def B23(x):
    return (1/2)*(2*x-3)


In [5]:
Be = np.array([[B21],[B22],[B23]])

Bet = np.transpose(Be)

print(Be)
print(Bet)

[[<function B21 at 0x000001B99F317C10>]
 [<function B22 at 0x000001B99F317CA0>]
 [<function B23 at 0x000001B99F3178B0>]]
[[<function B21 at 0x000001B99F317C10>
  <function B22 at 0x000001B99F317CA0>
  <function B23 at 0x000001B99F3178B0>]]
